In [1]:
import numpy as np
from devito import Grid, TimeFunction, Eq, solve, Operator, Constant
from sympy import nsimplify, symbols

# Viz
import matplotlib.pyplot as plt
%matplotlib inline

The `gcc` compiler isn't available on this system
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\sarth\Anaconda3\envs\srv-sim\lib\site-packages\devito\compiler.py", line 34, in sniff_compiler_version
    res = run([cc, "--version"], stdout=PIPE, stderr=DEVNULL)
  File "C:\Users\sarth\Anaconda3\envs\srv-sim\lib\subprocess.py", line 488, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\sarth\Anaconda3\envs\srv-sim\lib\subprocess.py", line 800, in __init__
    restore_signals, start_new_session)
  File "C:\Users\sarth\Anaconda3\envs\srv-sim\lib\subprocess.py", line 1207, in _execute_child
    startupinfo)
FileNotFoundError: [WinError 2] The system cannot find the file specified

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\sarth\Anaconda3\envs\srv-sim\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-6c397385de1b>", line 2, in <mod

TypeError: object of type 'NoneType' has no len()

In [ ]:
# initialize the grid
grid = Grid(shape=(100,), extent=(500e-9,)) # extent is in m

In [ ]:
grid.dimensions

In [ ]:
# carrier density is a function of time and space
n = TimeFunction(name="n", grid=grid, space_order=2)
n

In [ ]:
n.dt

In [ ]:
n.laplace

In [ ]:
n.forward

In [ ]:
# Create constants
D = Constant(name="D")
k1 = Constant(name="k1")
k2 = Constant(name="k2")

In [ ]:
# create the continuity equation
eq = Eq(n.dt, ((D * n.laplace) - (k1 * n) - (k2 * n**2)), subdomain=grid.interior)
eq

In [ ]:
# create the stencil
stencil = solve(eq, n.forward) # solve for the next time step

# create equation out of the stencil
eq_stencil = Eq(n.forward, stencil)

eq_stencil

In [ ]:
# initial conditions - carrier density at t=0

# TODO find a better/elegant way to incorporate the exponential decay condition
x = np.linspace(0, grid.extent[0], grid.shape[0])

n.data[0] = 1e15 * np.exp(-x/10e-9)

In [ ]:
plt.plot(n.data[0])

In [ ]:
# Create boundary conditions

x, = grid.dimensions
t = grid.stepping_dim

bc = [ Eq(n[t+1, 0], 1e15) ] # left

In [ ]:
# Variable declarations

nt = 50 # time steps

dt = 1e-9 # size of the steps in s

In [ ]:
# Create operator - create JIT-compiled and run C code

op = Operator([eq_stencil] + bc)

op(time=nt, dt=dt, D=0.9e-8, k1=1e7 * 1e-6, k2=1e-11 * 1e-12)

In [ ]:
plt.plot(n.data[0])